In [110]:
import pixiedust
from pixiedust.display.app import *
import pandas
from numpy import nan
import requests

@PixieApp
class PixieRosieApp:
    
    def setup(self):
        self.json_list = requests.get('http://clothing-history1.fyre.ibm.com:3000/api/com.biz.Clothing').json()
        self.header = list(self.json_list[0].keys())[1:]
        self.header[0] = "id"
        self.header[1] = "manufacturer"
        self.json_list = [list(j.values())[1:] for j in self.json_list]
        for counter, j in enumerate(self.json_list):
            j[1] = j[1][30:]
            self.json_list[counter] = j
        self.compare_list = list()
        self.cart_list = list()

    @route()
    def main_screen(self):
        return"""
            <nav class="navbar fixed-top navbar-default" style="font-size: 1.3em; font-weight: bold;">
              <div class="container-fluid">
                <ul class="nav navbar-nav">
                  <li style="border-right: 1px solid gray;"><a href="#">Home</a></li>
                  <li class="active" style="border-right: 1px solid gray;"><a href="#">Product Search</a></li>
                  <li style="border-right: 1px solid gray;"><a href="#">Product History</a></li>
                  <li style="border-right: 1px solid gray;"><a href="#">Sales Analytics</a></li>
                  <li style="border-right: 1px solid gray;"><a href="#">Customer Data Platform</a></li>
                  <li style="border-right: 1px solid gray;"><a href="#">
                    <i class="fa fa-shopping-cart" style="font-size: 15px;" aria-hidden="true"></i> 
                    {{this.cart_list|length}}
                    </a></li>
                </ul>
              </div>
            </nav>   
            <div id=target{{prefix}} class="well" style="max-height: 550px; overflow: auto;">
              <div class="panel panel-primary">
                <div class="panel-heading">Products</div>
                <div class="panel-body" style="max-height: 300px; overflow: auto;">
                    Text Search:
                    <input class="form-control" id="myInput" type="text">
                </div>
                <div class="panel-body" style="max-height: 300px; overflow: auto;">
                  <table class="table table-bordered table-striped">
                    <thead>
                      <th style="max-width: 40px"></th>
                      {%for name in this.header:%}
                          {% if loop.index != 4%}
                              <th title="{{name}}" style="text-align: left; text-overflow: ellipsis; overflow: hidden; white-space: nowrap; max-width: 100px">{{name}}</th>
                          {%endif%}
                      {%endfor%}
                      <th style="max-width: 110px">compliance</th>
                    </thead>
                    <tbody id="myTable">
                        {% for row in this.json_list %}
                            <tr>
                                <td style="text-align: left; max-width: 80px">
                                    <button class="btn btn-primary" style="font-size: 12px;" title="Show History" type="submit" pd_options="sp=true">
                                        <i class="fa fa-history" aria-hidden="true"></i>                                 
                                    </button>
                                    <button class="btn btn-primary" style="font-size: 12px;" title="Add to Cart" type="submit" pd_options="home=true">
                                        <i class="fa fa-cart-arrow-down" aria-hidden="true"></i> 
                                        <pd_script>
self.cart_list.append({{row}})
                                        </pd_script>
                                    </button>
                                    <button class="btn btn-primary" style="font-size: 12px;" title="Add to Saved" type="submit" pd_options="home=true">
                                        <i class="fa fa-plus" aria-hidden="true"></i> 
                                        <pd_script>
                                            self.compare_list.append({{row}})
                                        </pd_script>
                                    </button>
                                </td>
                                {% for col in row %}
                                    {% if loop.index != 4%}
                                        <td title="{{col}}" style="text-align: left; text-overflow: ellipsis; overflow: hidden; white-space: nowrap; max-width: 100px">{{col}}</td>
                                    {% elif loop.index == 2 %}
                                        <td title="{{col[:30]}}" style="text-align: left; text-overflow: ellipsis; overflow: hidden; white-space: nowrap; max-width: 100px">{{col[:30]}}</td>
                                    {%endif%}
                                {% endfor %}
                                <td style="text-align: left; max-width: 110px">
                                    {% if 'COMP1' in row[3] %}
                                        <button class="btn btn-primary" style="font-size: 12px;" title="EBA Certified" type="submit" pd_options="">
                                            <i class="fa fa-truck" aria-hidden="true"></i>                                      
                                        </button>
                                    {% endif %}
                                    {% if 'COMP2' in row[3] %}
                                        <button class="btn btn-primary" style="font-size: 12px;" title="FLA Compliant" onclick="window.open('http://www.fairlabor.org/affiliate/nike-inc')" target="_blank" type="submit" pd_options="">
                                            <i class="fa fa-users" aria-hidden="true"></i>                                  
                                        </button>
                                    {% endif %}
                                </td>
                            </tr>
                        {% endfor %}
                    </tbody>
                  </table>
                </div>
              </div>
           
            
            <div class="panel panel-primary">
                <div class="panel-heading">Saved</div>
                <div class="panel-body" style="max-height: 300px; overflow: auto;">
                  <table class="table table-bordered table-striped">
                    <thead>
                      {%if this.compare_list | length > 0%}
                          <th style="max-width: 100px"></th>
                          {%for name in this.header:%}
                              {% if loop.index != 4 %}
                                  <th title="{{name}}" style="text-align: left; text-overflow: ellipsis; overflow: hidden; white-space: nowrap; max-width: 100px">{{name}}</th>
                              {%endif%}
                          {%endfor%}
                          <th style="max-width: 110px">Compliance</th>
                      {% endif %}
                    </thead>
                        {% for row in this.compare_list %}
                            <tr>
                                <td style="text-align: left; max-width: 100px">
                                    <button class="btn btn-primary" style="font-size: 12px;" title="Show History" type="submit" pd_options="sp=true">
                                        <i class="fa fa-history" aria-hidden="true"></i>                                 
                                    </button>
                                    <button class="btn btn-primary" style="font-size: 12px;" title="Add to Cart" type="submit" pd_options="home=true">
                                        <i class="fa fa-cart-arrow-down" aria-hidden="true"></i> 
                                        <pd_script>
self.compare_list.remove({{row}})
self.cart_list.append({{row}})
                                        </pd_script>
                                    </button>
                                    <button class="btn btn-primary" style="font-size: 12px;" title="Remove From Saved" type="submit" pd_options="home=true">
                                        <i class="fa fa-times" aria-hidden="true"></i> 
                                        <pd_script>
                                            self.compare_list.remove({{row}})
                                        </pd_script>
                                    </button>
                                </td>
                                {% for col in row %}
                                    {% if loop.index != 4%}
                                        <td title="{{col}}" style="text-align: left; text-overflow: ellipsis; overflow: hidden; white-space: nowrap; max-width: 100px">{{col}}</td>
                                    {%endif%}
                                {% endfor %}
                                <td style="text-align: left; max-width: 110px">
                                    {% if 'COMP1' in row[3] %}
                                        <button class="btn btn-primary" style="font-size: 12px;" title="EBA Certified" type="submit" pd_options="">
                                            <i class="fa fa-truck" aria-hidden="true"></i>                                      
                                        </button>
                                    {% endif %}
                                    {% if 'COMP2' in row[3] %}
                                        <button class="btn btn-primary" style="font-size: 12px;" title="FLA Compliant" onclick="window.open('http://www.fairlabor.org/affiliate/nike-inc')" target="_blank" type="submit" pd_options="">
                                            <i class="fa fa-users" aria-hidden="true"></i>                                  
                                        </button>
                                    {% endif %}
                                </td>
                            </tr>
                        {% endfor %}
                  </table>
                </div>
              </div>
             </div>
            <script>
                $(document).ready(function(){
                  $("#myInput").on("keyup", function() {
                    var value = $(this).val().toLowerCase();
                    $("#myTable tr").filter(function() {
                      $(this).toggle($(this).text().toLowerCase().indexOf(value) > -1)
                    });
                  });
                });
            </script>
            """
    @route(sp="*")
    def supply_chain(self, sp):
        return"""
            <style>
              p {
                text-indent: 6.0em;
              }
            </style>
            <nav class="navbar fixed-top navbar-default" style="font-size: 1.3em; font-weight: bold;">
              <div class="container-fluid">
                <ul class="nav navbar-nav">
                  <li style="border-right: 1px solid gray;"><a href="#">Home</a></li>
                  <li style="border-right: 1px solid gray;"><a href="#">Product Search</a></li>
                  <li class="active" style="border-right: 1px solid gray;"><a href="#">Product History</a></li>
                  <li style="border-right: 1px solid gray;"><a href="#">Sales Analytics</a></li>
                  <li style="border-right: 1px solid gray;"><a href="#">Customer Data Platform</a></li>
                  <li style="border-right: 1px solid gray;"><a href="#">
                    <i class="fa fa-shopping-cart" style="font-size: 15px;" aria-hidden="true"></i> 
                    {{this.cart_list|length}}
                    </a></li>
                </ul>
              </div>
            </nav>  
            <div id=target{{prefix}} class="well" style="background-color: white">
                <img src="history.jpg" />
            </div>
            <input class="btn btn-primary" style="font-size: 12px;" pd_options="home=true" type="button" value="Back">
            """

In [111]:
PixieRosieApp().run(runInDialog='true')